In [171]:
# libs we'll use
import requests
from bs4 import BeautifulSoup

In [172]:
# server - weather.rambler.ru
serv_url = 'https://weather.rambler.ru/v-yaroslavle/'

In [173]:
# check date for format
def is_right_format(date: str) -> bool:
    if '.' in date:
        a, b = date.split('.')
        if a.isdigit() and b.isdigit():
            return True

# check date for range
def is_right_date(date: str) -> bool:
    day, month = date.split('.')
    if int(day) in range(0, 32) and int(month) in range(7, 9):
        return True

# get date - return link for date
def date_for_link(date: str) -> str:
    day, month = date.split('.')
    if int(month) == 7:
        month = 'july'
    elif int(month) == 8:
        month = 'august'
    return '{}-{}/'.format(day, month)

# link or None
def get_date_url(user_date: str) -> str:
    if is_right_format(user_date):
        if is_right_date(user_date):
            return date_for_link(user_date)

In [174]:
def get_url(date: str) -> str:
    url_date = get_date_url(date)
    if url_date:
        return serv_url + url_date

In [417]:
date = input()

url = get_url(date)
if url:
    r = requests.get(url)

12.08


In [418]:
assert r.status_code == 200

In [419]:
# we've got two page types: 
#     - detailed (close to us)
#     - generalized (feature)
# all the pages have main class w/ re: '_4pgy'
# dif:
#     - generalized include temp class w/ re: '_1HBR' 
# we'll get main class -> check for type -> get all the args

In [420]:
html = BeautifulSoup(r.content, 'html.parser')

In [425]:
# checking for class type
is_general_dict = {}
weather_dict = {}

def check_type():
    
    temp_lst = html.findAll('div', {'class': '_1HBR'})
    if temp_lst:
        is_general_dict['type'] = True
    else:
        is_general_dict['type'] = False
    
    
def get_main_class():
    
    re_main_class = re.compile('.*_4pgy.*')
    
    main_class = html.find('div', {'class': re_main_class})
    
def get_temp(general=True):
    if general:  # FIX IT  (get from check_type)
        weather_dict['temperature'] = html.find('div', {'class': '_1HBR'}).text
    else:
        re_temp_class = re.compile('.*_2ezK.*')
        temp_class = html.find('div', {'class': re_temp_class}) # 'Ночью14°Утром19°Днём24°Вечером22°'
        weather_dict['temperature'] = [int(number.group()) for number in ( 
                re.search(r'\d+', word) 
                for word in temp_class.text.split('°')) if number] 

def get_table():
    re_main_class = re.compile('.*2iSP.*')
    main_class = html.find('div', {'class': re_main_class})
    table_lst = re.findall('[А-Я|A-Z][^А-Я|A-Z]*', main_class.text)

    # if ветер in table:
    #     re_search
    # if давление in table...
    # if осадки in table:
    for param in table_lst:
        if 'Осадки' in param:
            weather_dict['precipation'] = re.search(r'\d+', param).group()
        if 'Ветер' in param:
            weather_dict['wind'] = re.search(r'\d+', param).group()
        if 'Давление' in param:
            weather_dict['pressure'] = re.search(r'\d+', param).group()
        if 'Восход' in param:
            weather_dict['sunrise'] = ':'.join(re.findall(r'\d+', param))
        if 'Закат' in param:
            weather_dict['sunset'] = ':'.join(re.findall(r'\d+', param))

In [426]:
get_temp()
get_table()
weather_dict

{'pressure': '754',
 'sunrise': '04:37',
 'sunset': '20:11',
 'temperature': '16°',
 'wind': '2'}

In [ ]:
# ! adapter for temperature ! HOW? let's think about it

# if general -> type_dict['general'] = True

# if type_dict['general']:
#     ...
# else:
#     ...